# Tutorial 6: Objects

In a previous tutorial we showed how to add an [engine-specific implementation](https://colab.research.google.com/github/eager-dev/eagerx_tutorials/blob/master/tutorials/pendulum/5_engine_implementation.ipynb) to an existing Object definition. In this tutorial, we will discuss how new [Objects](https://eagerx.readthedocs.io/en/master/guide/api_reference/object/index.html#eagerx.core.entities.Object) are defined.

The following will be covered:
- Defining a new [Object](https://eagerx.readthedocs.io/en/master/guide/api_reference/object/index.html#eagerx.core.entities.Object).

In the remainder of this tutorial we will go more into detail on this concept.

Furthermore, at the end of this notebook you will find an exercise.
For the exercise you will have to add/modify a couple of lines of code, which are marked by

```python

# START EXERCISE [BLOCK_NUMBER]

# END EXERCISE [BLOCK_NUMBER]
```

## Pendulum Swing-up

We will create an [Object](https://eagerx.readthedocs.io/en/master/guide/api_reference/object/index.html#eagerx.core.entities.Object) definition of the Pendulum, that was assumed to already be available in the previous tutorials.
The dynamics of the Pendulum object that is actuated by a DC motor are well known and we will define an [engine-specific implementation](https://eagerx.readthedocs.io/en/master/guide/api_reference/object/index.html#eagerx.core.entities.Object.example_engine) that uses to following set of ordinary differential equations (ODEs) for simulation:

$\mathbf{x} = \begin{bmatrix} \theta \\ \dot{\theta} \end{bmatrix} \\ \dot{\mathbf{x}} = \begin{bmatrix} \dot{\theta} \\ \frac{1}{J}(\frac{K}{R}u - mgl \sin{\theta} - b \dot{\theta} - \frac{K^2}{R}\dot{\theta})\end{bmatrix}$

with $\theta$ the angle w.r.t. upright position, $\dot{\theta}$ the angular velocity, $u$ the input voltage, $J$ the inertia, $m$ the mass, $g$ the gravitational constant, $l$ the length of the pendulum, $b$ the motor viscous friction constant, $K$ the motor constant and $R$ the electric resistance.

<img src="../figures/pendulum.GIF" width="480" />

## Activate GPU (Colab only)

When in Colab, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

## Notebook Setup

In order to be able to run the code, we need to install the *eagerx_tutorials* package.

In [2]:
try:
    import eagerx_tutorials
except ImportError:
    !{"echo 'Installing setuptools with pip.' && pip install setuptools==65.5.0 >> /tmp/setuptools_install.txt 2>&1"}
    !{"echo 'Installing eagerx-tutorials with pip.' && pip install eagerx-tutorials >> /tmp/eagerx_install.txt 2>&1"}

# Setup interactive notebook
# Required in interactive notebooks only.
from eagerx_tutorials import helper
helper.setup_notebook()

# Import eagerx
import eagerx
eagerx.set_log_level(eagerx.WARN)

Not running on CoLab.


## Let's get started


In previous tutorials we showed how to add [object](https://eagerx.readthedocs.io/en/master/guide/api_reference/object/index.html) entities to a [graph](https://eagerx.readthedocs.io/en/master/guide/api_reference/graph/graph.html). 
Apart from objects, graphs consist of nodes.
The difference between nodes and objects, is that nodes are engine-agnostic while objects have engine-specific implementations.
Objects are entities that can sense or act in the environment.
Nodes on the other hand can only process data.
A node could for example contain a classifier, PID controller or some sort of signal filter.
While we refer to the incoming and outcoming components of objects as `sensors` and `actuators`, a node has `inputs` and `outputs` since it cannot act or sense in the environment.

In this tutorial we will create a new [Object](https://eagerx.readthedocs.io/en/master/guide/api_reference/object/index.html#eagerx.core.entities.Object) called `Pendulum` with three sensors (`theta`, `theta_dot`, and `image`), a single actuator (`u`), and two states (`model_state`, `model_parameters`). You will be asked to finalize the implementation of the Pendulum node in the exercises at the end of this tutorial.

We can create a node by inheriting from the class [`Object`](https://eagerx.readthedocs.io/en/master/guide/api_reference/object/index.html).
This class has a single abstract methods we need to implement:

- [`make()`](https://eagerx.readthedocs.io/en/master/guide/api_reference/node/node.html#eagerx.core.entities.Node.make): Makes the parameter specification of the node. We must use the register decorator to register all `sensors`, `acutators`, and `states` with their corresponding [Space](https://eagerx.readthedocs.io/en/master/guide/api_reference/utilities/space.html). 

In [3]:
from typing import List
from eagerx import Object, Space, register
from eagerx.core.specs import ObjectSpec


class Pendulum(Object):
    @classmethod
    @register.sensors(
        theta=Space(low=-999.0, high=999.0, shape=(), dtype="float32"),
        theta_dot=Space(low=-999.0, high=999.0, shape=(), dtype="float32"),
        # START EXERCISE 1.1
        # Define `x` as a sensor.
        x=Space(low=[-999, -999], high=[999, 999], shape=(2,), dtype="float32"),  # Solution
        # END EXERCISE 1.1
        image=Space(dtype="uint8"),
    )
    @register.actuators(u=Space(low=[-2], high=[2], dtype="float32"))
    @register.engine_states(
        model_state=Space(low=[-3.14, -9], high=[3.14, 9], dtype="float32"),
    )
    def make(
        cls,
        name: str,
        actuators: List[str] = None,
        sensors: List[str] = None,
        states: List[str] = None,
    ):
        """Make the specification of the Pendulum.

        :param name: Name of the object.
        :param actuators: The selected actuators. Must be a subset of the registered actuators.
                          - u := DC motor voltage.
        :param sensors: The selected sensors. Must be a subset of the registered sensors.
                        - theta := angle of the pendulum wrt upward position.
                        - theta_dot := angular velocity of the pendulum.
                        - image := rendered rgb image of the pendulum.
        :param states: The selected states. Must be a subset of the registered states.
                       - model_state := allows resetting of the initial state, defined as the angle and angular velocity.
                       - model_parameters := allows resetting all ODE parameters [J, m, l, b, K, R, c, d].
        """
        spec = cls.get_specification()

        # Set objects ("agnostic") config parameters
        spec.config.name = name
        spec.config.sensors = sensors  # Selected sensors (can be subset of registered sensors)
        spec.config.actuators = actuators  # Selected actuators (can be subset of registered actuators)
        spec.config.states = states  # Selected states (can be subset of registered states)

        # Set rates of all sensor and actuators (here fixed at 30/15 Hz).
        spec.sensors.theta.rate = 30.  # Hz
        spec.sensors.theta_dot.rate = 30.  # Hz
        spec.sensors.image.rate = 15.  # Hz
        spec.actuators.u.rate = 30.  # Hz
        # START EXERCISE 1.2
        # Set the rate of sensor `x`
        spec.sensors.x.rate = 30.  # Solution
        # END EXERCISE 1.2
        return spec


We can make the parameter specification (i.e. `spec`) of the newly defined Object `Pendulum` and add it to a [graph](https://eagerx.readthedocs.io/en/master/guide/api_reference/graph/graph.html).

In [4]:
# Print info on newly defined object
Pendulum.info()

# Make pendulum parameter specification
# START EXERCISE 1.4
# Select sensor `x` instead of sensors `theta` and `theta_dot`.
# pendulum = Pendulum.make("pendulum", sensors=["theta", "theta_dot", "image"], actuators=["u"], states=["model_state"])
pendulum = Pendulum.make("pendulum", sensors=["x", "image"], actuators=["u"], states=["model_state"])  # Solution
# START EXERCISE 1.4

# Initialize empty graph with pendulum
graph = eagerx.Graph.create(objects=pendulum)

# Connect the pendulum to an action and observation
graph.connect(action="voltage", target=pendulum.actuators.u)
# START EXERCISE 1.5
# Connect sensor `x` as an observation.
# graph.connect(source=pendulum.sensors.theta, observation="angle")
# graph.connect(source=pendulum.sensors.theta_dot, observation="angular_velocity")
graph.connect(source=pendulum.sensors.x, observation="x")  # Solution
# START EXERCISE 1.5

# Render image
graph.render(source=pendulum.sensors.image, rate=15)

   entity_type: `Pendulum`
   module: `__main__`
   file: <unknown>

Supported engines: <Nothing registered>

Make this spec with:
   spec = Pendulum.make(name: str, actuators: List[str] = None, sensors: List[str] = None, states: List[str] = None)

class Pendulum:
   make(name: str, actuators: List[str] = None, sensors: List[str] = None, states: List[str] = None):
      sensors:
       - theta: Space(-999.0, 999.0, (), float32)
       - theta_dot: Space(-999.0, 999.0, (), float32)
       - x: Space([-999. -999.], [999. 999.], (2,), float32)
       - image: Space(uint8)
      actuators:
       - u: Space([-2.], [2.], (1,), float32)
      engine_states:
       - model_state: Space([-3.14 -9.  ], [3.14 9.  ], (2,), float32)
      docs:
         Make the specification of the Pendulum.

                 :param name: Name of the object.
                 :param actuators: The selected actuators. Must be a subset of the registered actuators.
                                   - u := DC motor v

To truly initialize an environment with this graph and a selected [engine](https://eagerx.readthedocs.io/en/master/guide/api_reference/engine/index.html), every object in this graph must have an [engine-specific implementation](https://colab.research.google.com/github/eager-dev/eagerx_tutorials/blob/master/tutorials/pendulum/5_engine_implementation.ipynb) for that selected engine. However, `Pendulum.info()` indicates: `Supported engines: <Nothing registered>`. In other words, there are no engine implementeations registered yet. This is of course correct, because we essentially only defined an "agnostic" interface for the Pendulum (so that it can be added to the graph), but we have not specified how the sensors, actuators, and states should actually be implemented given an engine. Therefore, we will now add an engine-specific implementation for the [OdeEngine](https://github.com/eager-dev/eagerx_ode/blob/master/eagerx_ode/engine.py). We will follow the same procedure as in a previous [previous tutorial](https://colab.research.google.com/github/eager-dev/eagerx_tutorials/blob/master/tutorials/pendulum/5_engine_implementation.ipynb) by defining:

- `ode_engine()` (arbitrary naming): An [engine-specific implementation](https://colab.research.google.com/github/eager-dev/eagerx_tutorials/blob/master/tutorials/pendulum/5_engine_implementation.ipynb) of the `Pendulum` for the [OdeEngine](https://github.com/eager-dev/eagerx_ode/blob/master/eagerx_ode/engine.py).

In [5]:
from eagerx import register
from eagerx_ode.engine import OdeEngine
from eagerx.core.graph_engine import EngineGraph

# Print info on the OdeEngine to find out what parameters can be set.
# That is, all arguments (except for `spec`) to the add_object method.
OdeEngine.info("add_object")


# This decorator registers engine implementation with default object_params
@register.engine(OdeEngine, entity=Pendulum)  
def ode_engine(spec: ObjectSpec, graph: EngineGraph):
    """Engine-specific implementation (OdeEngine) of the object."""
    # Set paths to ODE & gradient (Jacobian) functions.
    spec.engine.ode = "eagerx_tutorials.pendulum.pendulum_ode/pendulum_ode"
    spec.engine.Dfun = "eagerx_tutorials.pendulum.pendulum_ode/pendulum_dfun"

    # Set params of pendulum ode: [J, m, l, b, K, R, c, d].
    spec.engine.ode_params = [0.00016, 0.051, 0.042, 1.4e-05, 0.033, 7.7, 0.0010, 165.]

    # Create engine_states (no agnostic states defined in this case)
    from eagerx_ode.engine_states import OdeEngineState
    spec.engine.states.model_state = OdeEngineState.make()

    # Import engine nodes
    from eagerx_tutorials.pendulum.engine_nodes import FloatOutput
    from eagerx_ode.engine_nodes import OdeOutput, OdeInput, OdeRender, ActionApplied
    
    # Create sensor engine nodes
    x = OdeOutput.make("x", rate=spec.sensors.theta.rate, process=2)

    # For didactic purposes, we create two sensors, i.e. one with angle and one with angular velocity.
    # We could also have created a sensor that contains both, but in this way it is more clear which sensor
    # contains what information.
    theta = FloatOutput.make("theta", rate=spec.sensors.theta.rate, idx=0)
    theta_dot = FloatOutput.make("theta_dot", rate=spec.sensors.theta_dot.rate, idx=1)

    render_fn = "eagerx_tutorials.pendulum.pendulum_render/pendulum_render_fn"
    image = OdeRender.make("image", render_fn=render_fn, rate=spec.sensors.image.rate, process=2, shape=[480, 480, 3])

    # Create actuator engine node
    u = OdeInput.make("u", rate=spec.actuators.u.rate, process=2, default_action=[0])

    # Add all engine nodes
    graph.add([x, theta, theta_dot, image, u])

    # Connect: theta
    graph.connect(source=x.outputs.observation, target=theta.inputs.observation_array)
    graph.connect(source=theta.outputs.observation, sensor="theta")

    # Connect: theta_dot
    graph.connect(source=x.outputs.observation, target=theta_dot.inputs.observation_array)
    graph.connect(source=theta_dot.outputs.observation, sensor="theta_dot")
    
    # Connect: x
    # START EXERCISE 1.3
    # Connect sensor `x` to the `observation` output of node `x`.
    graph.connect(source=x.outputs.observation, sensor="x")
    # END EXERCISE 1.3
    
    # Connect: u
    graph.connect(actuator="u", target=u.inputs.action)
    
    # Connect: image
    graph.connect(source=x.outputs.observation, target=image.inputs.observation)
    graph.connect(source=u.outputs.action_applied, target=image.inputs.action_applied, skip=True)
    graph.connect(source=image.outputs.image, sensor="image")


   entity_type: `OdeEngine`
   module: `eagerx_ode.engine`
   file: `/home/bas/.cache/pypoetry/virtualenvs/eagerx-tutorials-uyJhHlY9-py3.8/lib/python3.8/site-packages/eagerx_ode/engine.py`

Make this spec with:
   spec = OdeEngine.make(rate: float, sync: Union[bool, NoneType] = True, process: Union[int, NoneType] = 1, real_time_factor: Union[float, NoneType] = 0, simulate_delays: Union[bool, NoneType] = True, log_level: Union[int, NoneType] = 40, rtol: float = 2e-08, atol: float = 2e-08, hmax: float = 0.0, hmin: float = 0.0, mxstep: int = 0)

class OdeEngine:
   add_object(self, spec, ode: str = None, Dfun: Union[str, NoneType] = None, ode_params: Union[List, NoneType] = None):
      engine_config: <Nothing registered>
      docs: <Not available>




Then we proceed as usual, by creating the engine specification of the [OdeEngine](https://github.com/eager-dev/eagerx_ode/blob/master/eagerx_ode/engine.py) and initialize an environment with it like we did in previous tutorials.

In [6]:
from typing import Dict
import numpy as np


class PendulumEnv(eagerx.BaseEnv):
    def __init__(self, name: str, rate: float, graph: eagerx.Graph, engine: eagerx.Engine):
        """Initializes an environment with EAGERx dynamics.

        :param name: The name of the environment. Everything related to this environment
                     (parameters, topics, nodes, etc...) will be registered under namespace: "/[name]".
        :param rate: The rate (Hz) at which the environment will run.
        :param graph: The graph consisting of nodes and objects that describe the environment's dynamics.
        :param engine: The physics engine that will govern the environment's dynamics.
        """
        # Make the backend specification
        from eagerx.backends.single_process import SingleProcess
        backend = SingleProcess.make()
        
        self.eval = eval
        
        # Maximum episode length
        self.max_steps = 100
        
        # Step counter
        self.steps = None
        super().__init__(name, rate, graph, engine, backend, force_start=True)
    
    def step(self, action: Dict):
        """A method that runs one timestep of the environment's dynamics.

        :params action: A dictionary of actions provided by the agent.
        :returns: A tuple (observation, reward, done, info).

            - observation: Dictionary of observations of the current timestep.

            - reward: amount of reward returned after previous action

            - done: whether the episode has ended, in which case further step() calls will return undefined results

            - info: contains auxiliary diagnostic information (helpful for debugging, and sometimes learning)
        """
        # Take step
        observation = self._step(action)
        self.steps += 1
        
        # Get angle and angular velocity
        # Take first element because of window size (covered in other tutorial)
        # START EXERCISE 1.6
        # Extract th, thdot from observation `x`.
        # th = observation["angle"][0]
        # thdot = observation["angular_velocity"][0]
        th, thdot = observation["x"][0]  # Solution
        # END EXERCISE 1.6
        
        # Convert from numpy array to float
        u = float(action["voltage"])

        # Calculate cost
        # Penalize angle error, angular velocity and input voltage
        cost = th**2 + 0.1 * (thdot / (1 + 10 * abs(th))) ** 2 + 0.01 * u ** 2  

        # Determine when is the episode over
        # currently just a timeout after 100 steps
        done = self.steps > self.max_steps

        # Set info, tell the algorithm the termination was due to a timeout
        # (the episode was truncated)
        info = {"TimeLimit.truncated": self.steps > self.max_steps}
        
        return observation, -cost, done, info
    
    def reset(self) -> Dict:
        """Resets the environment to an initial state and returns an initial observation.

        :returns: The initial observation.
        """
        # Determine reset states
        states = self.state_space.sample()
            
        # Perform reset
        observation = self._reset(states)

        # Reset step counter
        self.steps = 0
        return observation

    
# Make the engine
engine = OdeEngine.make(rate=30.)
    
# Initialize Environment
env = PendulumEnv(name="PendulumEnv", rate=30, graph=graph, engine=engine)

# Print action & observation space
print("action_space: ", env.action_space)
print("observation_space: ", env.observation_space)

[WARN]: Backend 'SINGLE_PROCESS' does not support multiprocessing, so all nodes are launched in the ENVIRONMENT process.
action_space:  Dict(voltage:Space([-2.], [2.], (1,), float32))
observation_space:  Dict(x:Box([[-999. -999.]], [[999. 999.]], (1, 2), float32))


Finally, we train the agent using [Stable Baselines3](https://stable-baselines3.readthedocs.io/en/master/), again similar to the preceding tutorials.

In [7]:
import stable_baselines3 as sb3
from stable_baselines3.common.env_checker import check_env
from eagerx.wrappers import Flatten
from gym.wrappers.rescale_action import RescaleAction

# Stable Baselines3 expects flattened actions & observations
# Convert observation and action space from Dict() to Box(), normalize actions
env = Flatten(env)
env = RescaleAction(env, min_action=-1.0, max_action=1.0)

# Check that env follows Gym API and returns expected shapes
check_env(env)

# Toggle render
env.render("human")

# Initialize learner
model = sb3.SAC("MlpPolicy", env, verbose=1)

# Train for 1 minute (sim time)
model.learn(total_timesteps=int(150 * 30))

env.shutdown()

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 101      |
|    ep_rew_mean     | -997     |
| time/              |          |
|    episodes        | 4        |
|    fps             | 92       |
|    time_elapsed    | 4        |
|    total_timesteps | 404      |
| train/             |          |
|    actor_loss      | 18.6     |
|    critic_loss     | 2.66     |
|    ent_coef        | 0.914    |
|    ent_coef_loss   | -0.145   |
|    learning_rate   | 0.0003   |
|    n_updates       | 303      |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 101      |
|    ep_rew_mean     | -959     |
| time/              |          |
|    episodes        | 8        |
|    fps             | 84       |
|    time_elapsed    | 9        |
|    total_timesteps | 808      |
| train/             

# Exercise

In this exercise you will add a new sensor to the object definition of the underactuated pendulum.

For this exercise, you will need to modify or add some lines of code in the cells above.
These lines are indicated by the following comments:

```python
# START EXERCISE [BLOCK_NUMBER]

# END EXERCISE [BLOCK_NUMBER]
```

However, feel free to play with the other code as well if you are interested.
We recommend you to restart and run all code after each section (in Colab there is the option *Restart and run all* under *Runtime*).

## 1. Define a new sensor `x`

For didactic purposes, we initially created two sensors, i.e. one with angle and one with angular velocity measurements.
However, we could also have created a sensor that contains both. 
In fact, in the engine-specific implementation we actually alreade use engine node `x` that extracts the ODE state and outputs that to the two engine nodes `theta` and `theta_dot`.
Therefore, as an exercise we will define a new sensor `x`, that will contain both `theta` and `theta_dot` measurements, and the output of engine node `x` to implement that sensor.

1.1 Add a sensor `x` to the `@register.sensor` decorator with `low=[-999, -999]` and `high=[999, 999]`.

1.2 Set the `rate` of sensor `x` to `30`.

1.3 Connect `x.outputs.observation` to sensor `x` in the engine implementation. You **don't** have to disconnect `theta` and `theta_dot`. By not selecting them when making the parameter specification, eagerx will figure out that these nodes are not needed and will forgo on their initialization.

1.4 Replace the currently selected `theta` and `theta_dot` with sensor  `x`.

1.5 Instead of connecting `theta` and `theta_dot` as observations `angle` and `angular_velocity` respectively, we will connect `x` as the sole observation.

1.6 Now that we have selected `x` as an observation, modify `PendulumEnv.step()` such that observation `x` is unpacked into `th` and `thdot`. 